In [1]:
# %pip install transformers
# %pip install hazm

In [2]:
#load model
import tensorflow as tf
import numpy as np
import itertools
from transformers import TFAutoModelForMaskedLM, AutoTokenizer
import time

model_name = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForMaskedLM.from_pretrained(model_name, output_hidden_states=False, output_attentions=False)

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [3]:
def calculate_sentence_score(sentence_tokens,k=1000,epsilon=0.000001):
    sentence_score = 1000000000000
    sentence = ' '.join(sentence_tokens)
    for i,token in enumerate(sentence_tokens):
        found = False
        masked_text = ' '.join(sentence_tokens[:i])+' [MASK] '+' '.join(sentence_tokens[i+1:])
        input_ids = tokenizer.encode(masked_text, return_tensors="tf")
        logits = logits = model.__call__(input_ids)[0]
        mask_token_index = tf.where(input_ids == tokenizer.mask_token_id)[0, 1]
        mask_token_logits = logits[0, mask_token_index, :]
        top_k_values, top_k_indices = tf.math.top_k(mask_token_logits, k=k)
        predicted_token_indices = top_k_indices.numpy()
        probabilities = tf.nn.softmax(top_k_values).numpy()
        predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_indices)
        for j,predicted_token in enumerate(predicted_tokens):
            if predicted_token == token:
                sentence_score *= probabilities[j]
                found = True
                break
        if not found:
            sentence_score *= epsilon
        # print(token)
        # for i in range(k):
        #     print(predicted_tokens[i],probabilities[i])
        # print('----')
    return sentence_score

In [4]:
def clean_permutations(permutations):
    return permutations

In [5]:
def find_ordered_sentence(sentence):
    tokens = tokenizer.tokenize(sentence)
    permutations = list(itertools.permutations(tokens))
    cleaned_permutations = clean_permutations(permutations)
    scores = []
    for i,permutation in enumerate(cleaned_permutations):
        permutation_sentence = ' '.join(permutation)
        score = calculate_sentence_score(list(permutation))
        print(permutation_sentence)
        print(score)
        scores.append((permutation_sentence,score))
        print(i+1,'/',len(permutations))
    scores.sort(key=lambda x: x[1], reverse=True)
    for score in scores:
        print(score)
    return scores[0][1]

In [6]:
#main code
find_ordered_sentence('ماشین من قرمز است')

ماشین من قرمز است
1301.1877311779792
1 / 24
ماشین من است قرمز
9.540694058561872
2 / 24
ماشین قرمز من است
1.260887949989728
3 / 24
ماشین قرمز است من
0.05171480819283033
4 / 24
ماشین است من قرمز
3.0667185302706295e-07
5 / 24
ماشین است قرمز من
5.424627092658384e-07
6 / 24
من ماشین قرمز است
1.6646597236552263e-05
7 / 24
من ماشین است قرمز
0.0003028538145117322
8 / 24
من قرمز ماشین است
4.507739903831098e-05
9 / 24
من قرمز است ماشین
4.582357173422339e-06
10 / 24
من است ماشین قرمز
0.000767257715028859
11 / 24
من است قرمز ماشین
2.952986497243013e-07
12 / 24
قرمز ماشین من است
0.03649829303855068
13 / 24
قرمز ماشین است من
8.991581962255245e-05
14 / 24
قرمز من ماشین است
0.002887629940165474
15 / 24
قرمز من است ماشین
7.746683887098179e-05
16 / 24
قرمز است ماشین من
0.5908915839240025
17 / 24
قرمز است من ماشین
5.656760871917895e-06
18 / 24
است ماشین من قرمز
0.0011202331024374073
19 / 24
است ماشین قرمز من
0.03762213100317759
20 / 24
است من ماشین قرمز
8.888369905607896e-06
21 / 24
است من قرمز ماشین
1.2

1301.1877311779792